In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


s:\Explainable AI\myenv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [15]:
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# data\Vegetable Images\test

import os

path = r"data\Vegetable Images\test"

classes = [
    d for d in os.listdir(path)
    if os.path.isdir(os.path.join(path, d))
]

print("Number of classes:", len(classes))
print("Class names:", classes)


In [17]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(15, activation='softmax')  # 15 classes
])
 

In [19]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [20]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)


In [21]:
train_data = train_gen.flow_from_directory(
    'data/Vegetable Images/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_gen.flow_from_directory(
    'data/Vegetable Images/validation',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)


Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4758 - loss: 1.7699

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img("test.jpg", target_size=(224,224))
img = image.img_to_array(img) / 255.0
img = np.expand_dims(img, axis=0)

prediction = model.predict(img)
print(prediction)
